referenfce: https://github.com/ukairia777/finance_sentiment_corpus

학습 데이터와 검증 데이터를 비교하여 모델의 성능을 평가할 수 있다. \
정확도는 검증 데이터의 정확도를 말한다. \
검증 데이터의 loss와 학습데이터와 검증데이터의 차이를 통해 과적합을 판단할 수 있다. \
검증데이터의 train과 test데이터의 비율, 학습 데이터의 train과 test데이터의 비율을 변경하여 정확도를 높일 수 있다.\
데이터 셋의 감정 비율을 정확하게 나눠서 정확도를 높일 수 있다. 

### Data Load | Train, Valid, Test 데이터 나누기

In [ ]:
# !pip install transformers 
# !pip install datasets

In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
# 데이터 로드 >> 데이터 프레임에 저장 
# df = pd.read_csv('./model/data1(p1)+data2_re.csv',index_col=0) 
# print('샘플의 개수:', len(df))

In [ ]:
# Pandas를 사용하여 로드하거나 Hugging Face의 datasets 라이브러리를 사용하여 로드할 수 있으며, 
# 필요에 따라 적절한 방식을 선택하면 됩니다.

# load_dataset 모듈 활용 
from datasets import load_dataset

all_data = load_dataset(
    "csv",
    data_files={
        "train": "./data/data1(p1)+data2_re.csv"
    }
)

# 현재 train에 모든 데이터가 저장됨 
# load_dataset() 허깅페이스 사에서 만든 transformer library 

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 942.96it/s]


Dataset csv downloaded and prepared to C:/Users/SBAuser/.cache/huggingface/datasets/csv/default-8cc7bfd0870e4e86/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 487.99it/s]


In [ ]:
# dataset 모듈 활용, train/test set 분류 
# train_test_split()

cs = all_data['train'].train_test_split(0.2)
# 8:2 비율로 훈련 데이터, 검증 데이터 구분, 저장 
train_cs = cs['train']
test_cs = cs['test']

In [ ]:
train_cs

Dataset({
    features: ['labels', 'kor_sentence'],
    num_rows: 32983
})

In [ ]:
test_cs

Dataset({
    features: ['labels', 'kor_sentence'],
    num_rows: 8246
})

In [ ]:
# 검증 데이터 생성하기 
# 훈련용 데이터를 다시 8:2로 구분, 저장 

cs = train_cs.train_test_split(0.2)
train_cs = cs['train']
valid_cs = cs['test']

In [ ]:
# 훈련 데이터 
print(train_cs)
print()
print(valid_cs)
print()
print(test_cs)

Dataset({
    features: ['labels', 'kor_sentence'],
    num_rows: 26386
})

Dataset({
    features: ['labels', 'kor_sentence'],
    num_rows: 6597
})

Dataset({
    features: ['labels', 'kor_sentence'],
    num_rows: 8246
})


In [ ]:
print('두번째 샘플 출력:', train_cs['kor_sentence'][1])
print('두번째 샘플의 레이블 출력:', train_cs['labels'][1])

두번째 샘플 출력: 온국민이 억울하다 얘
두번째 샘플의 레이블 출력: 2


### 데이터 셋 전처리

In [ ]:
import numpy as np 
import pandas as pd
import random 
import time
import datetime 
from tqdm import tqdm 

import csv 
import os 

import tensorflow as tf 
import torch 

In [ ]:
# BERT 사용하기 위한 모듈 불러오기 

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
# 패딩(padding) 위한 모듈 불러오기 

from tensorflow.keras.preprocessing.sequence import pad_sequences

# 전처리 및 평가지표 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, hamming_loss

In [ ]:
# BERT 구조 
# [CLS] sequence [SEP] 구조

# 훈련 데이터, 검증 데이터, 테스트 데이터를 BERT 구조로 변환 

train_sentences = list(map(lambda x: '[CLS]' + str(x) + '[SEP]', train_cs['kor_sentence']))
validation_sentences = list(map(lambda x: '[CLS]' + str(x) + '[SEP]', valid_cs['kor_sentence']))
test_sentences = list(map(lambda x: '[CLS]' + str(x) + '[SEP]', test_cs['kor_sentence']))

In [ ]:
train_labels = train_cs['labels']
validation_labels = valid_cs['labels']
test_labels = test_cs['labels']

In [ ]:
test_sentences[:5]

['[CLS]아내의 병증이 점점 심해져 누가 항상 옆에서 보살펴 줘야 하는데 방법이 없네.[SEP]',
 '[CLS]내가 앓고 있는 공황장애 때문에 부모님이 많이 싸우셔.[SEP]',
 '[CLS]건강하게 몸 관리 잘 하시고언제나 그랬듯이 최선을 다하는 모습이 보물 같습니다[SEP]',
 '[CLS]언젠가 어디선가 좋은날 계속 사랑하고 있길 바라며안녕 도깨비 [SEP]',
 '[CLS]무상기증을 당연시하지마라[SEP]']

In [ ]:
# 행복 = 0, 슬픔 = 1, 분노 = 2 

test_labels[:5]

[2, 2, 1, 2, 1]

BERT 토큰나이저 이용한 전처리 

- BERT 사용하기 위해서는 tokenizer와 model이 반드시 mapping 되는 관계여야만 함 >> 아래의 이름에 들어가는 모델이름은 반드시 동일해야 함 
- BertTokenizer.from_pretrained('모델이름') 
- BertForSequenceClassification.from_pretrained('모델이름') 

- Tokenizer의 역할은 내부적으로 vocabulary를 갖고 있어서 정수 인코딩을 수행해주는 모듈

In [ ]:
# kor-bert 중 하나니 'klue/bert-base'를 사용 
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

# klue/bert-base는 Hugging Face의 Transformers 라이브러리에서 제공하는 한국어 BERT 모델 중 하나입니다. 
# 해당 모델을 사용하기 위해 BertTokenizer.from_pretrained를 사용하여 토크나이저를 불러올 수 있습니다.

# 위 코드를 실행하면 klue/bert-base 모델의 사전을 기반으로 한 토크나이저 객체가 생성됩니다. 
# 이 토크나이저를 사용하여 텍스트를 토큰화하고, 토큰을 정수로 변환하거나 정수를 토큰으로 변환할 수 있습니다.

In [ ]:
MAX_LEN = 64
# pre-trained model : 최대 길이 512
# fine-tuning 할 때 데이터 셋 길이 128

def data_to_tensor(sentences, labels): 
   # 정수 인코딩 과정, 각 텍스트를 토큰화한 후에 vocabulary에 mapping 되는 정수 시퀀스로 변환 
   # 예) ['안녕하세요'] >> ['안','녕','하세요'] >> [231,52,45]
   tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
   input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

   # pad_sequences: padding 위한 모듈. 주어진 max_len(최대길이)위에 뒤에 0으로 채움 
   # 예) [231,52,45] >> [231,52,45,0,0,0]

   input_ids = pad_sequences(input_ids, maxlen = MAX_LEN, dtype='long', truncating='post', padding='post')
   # truncating은 시퀀스가 최대 길이를 초과할 경우 어느 부분을 삭제할 것인지를 설정합니다. 
   # 여기서는 'post'로 설정되어 시퀀스의 뒷부분이 잘리게 됩니다. 
   # padding은 패딩에 사용할 값으로, 'post'로 설정하면 시퀀스의 뒷부분이 패딩 값으로 채워집니다.
      
   attention_masks =[]

   for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)
      # float(i > 0)를 사용하여 패딩이 아닌 토큰에 대해서는 1, 패딩 토큰에 대해서는 0으로 값을 설정합니다.

   # 각각 PyTorch의 torch.tensor로 변환
   tensor_inputs = torch.tensor(input_ids)
   tensor_labels = torch.tensor(labels)
   tensor_masks = torch.tensor(attention_masks)

   return tensor_inputs, tensor_labels, tensor_masks


In [ ]:
# 훈련 데이터, 검증데이터, 테스트 데이터에 대해서 data_to_tensor 함수를 활용
# 정수 인코딩된 데이터, 레이블, 어텐션 마스크를 얻음 

train_inputs, train_labels, train_masks = data_to_tensor(train_sentences, train_labels)
validation_inputs, validation_labels, validation_masks = data_to_tensor(validation_sentences, validation_labels)
test_inputs, test_labels, test_masks = data_to_tensor(test_sentences, test_labels)

C:\Users\SBAuser\AppData\Local\Temp\ipykernel_19944\3465040229.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor_labels = torch.tensor(labels)


In [ ]:
print(train_inputs[0])
print()
print(train_labels[0])
print()
print(train_masks[0])

tensor([    2,  7740,  1513,  6186, 27321,  2059,  2088,  1041,  2200,  4074,
         2085,  1295,  1415,  2069,  3681,  2200,  4093,  2182,     3,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0], dtype=torch.int32)

tensor(1)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [ ]:
tokenizer.decode

<bound method PreTrainedTokenizerBase.decode of BertTokenizer(name_or_path='klue/bert-base', vocab_size=32000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)>

In [ ]:
tokenizer.decode([0])

'[PAD]'

In [ ]:
tokenizer.decode([3])

'[SEP]'

batch_size = 32 
- pytorch의 DataLoader이용, 변환 
- DataLoader: batch 단위로 데이터를 꺼내올 수 있도록 하는 모듈 

In [ ]:
batch_size = 32

#  PyTorch의 TensorDataset, RandomSampler, 그리고 DataLoader를 사용하여 훈련 데이터를 미니배치로 나눈다.

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
# train_data를 입력으로 받아서 무작위로 샘플링하는 랜덤 샘플러를 생성
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
# DataLoader: train_data, train_sampler, 그리고 batch_size를 입력으로 받아서 데이터 로더를 생성합니다. 
# 이 데이터 로더는 미니배치로 데이터를 로드하는 역할을 합니다. 
# train_dataloader를 통해 반복문을 사용하여 미니배치 단위로 데이터를 로드하고 모델을 훈련할 수 있습니다.


In [ ]:
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
print('훈련 데이터 크기:', len(train_labels))
print('검증 데이터 크기:', len(validation_labels))
print('테스트 데이터 크기:', len(test_labels))

훈련 데이터 크기: 26386
검증 데이터 크기: 6597
테스트 데이터 크기: 8246


In [ ]:
# GPU 정상 세팅이 되었는지 확인 

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 3070


### 모델 로드하기 

- BERT 사용, 텍스트 분류 아키텍처
  - BertForSequenceClassification.from_pretrained('모델명') 
  - label 개수: num_labels

In [ ]:
num_labels = 3

model = BertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=num_labels)
model.cuda()

#  "klue/bert-base" 사전 훈련된 BERT 모델을 불러와 시퀀스 분류 작업을 위한 모델 객체를 생성하고, 
# 해당 모델을 GPU로 이동
#  "klue/bert-base"는 사전 훈련된 BERT 모델의 이름
# 모델을 GPU로 이동시키기 위해 model.cuda()를 호출
# 이를 통해 모델의 모든 매개변수와 버퍼가 GPU 메모리에 할당

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# 최적화(optimizer) 선택 

optimizer = AdamW(model.parameters(),
            lr= 2e-5,
            eps=1e-8)

# 학습 반복 횟수 설정 (epochs)

epochs = 2
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                num_warmup_steps = 0, 
                                num_training_steps = total_steps)

c:\Users\SBAuser\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# 훈련 과정에서 경과 시간을 형식화
def format_time(elapsed): 
   elapsed_rounded = int(round(elapsed))
   return str(datetime.timedelta(seconds=elapsed_rounded)) 
   # hh:mm:ss 

In [ ]:
def metrics(predictions, labels): 
   y_pred = predictions
   y_true = labels

   # 사용 가능한 메트릭들을 사용함 
   accuracy = accuracy_score(y_true, y_pred)
   # accuracy_score 함수
   # y_true와 y_pred 두 개의 배열을 입력으로 받아 정확도를 계산
   f1_macro_average = f1_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0) 
   # 'macro'는 클래스별 F1 스코어의 평균을 계산
   f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro', zero_division=0) 
   # 'micro'는 전체 예측 결과에 대한 F1 스코어를 계산
   f1_weighted_average = f1_score(y_true=y_true, y_pred=y_pred, average='weighted', zero_division=0)
   # 'weighted'는 클래스별 샘플 수로 가중 평균된 F1 스코어를 계산
   # zero_division 매개변수는 분모가 0일 경우 처리 방법을 지정
   
   # metric 결과에 대해서도 return 

   metrics = {'accuracy': accuracy, 
              'f1_macro': f1_macro_average,
              'f1_micro': f1_micro_average,
              'f1_weighted': f1_weighted_average
              }

   return metrics

### 모델 학습 

In [ ]:
# random seed 값 설정 

seed_val = 777
# seed_val에 원하는 시드 값을 지정합니다.
random.seed(seed_val)
# 파이썬의 random 모듈에 시드 값을 설정
np.random.seed(seed_val)
# NumPy의 랜덤 모듈에 시드 값을 설정
torch.manual_seed(seed_val)
# PyTorch의 랜덤 모듈에 시드 값을 설정
torch.cuda.manual_seed_all(seed_val)
# CUDA 디바이스를 사용하는 경우, 모든 CUDA 디바이스에 시드 값을 설정


model.zero_grad()
# model.zero_grad()는 모델의 모든 가중치에 대한 그래디언트를 0으로 초기화하는 작업을 수행
# 딥러닝 모델을 학습할 때, 역전파(backpropagation) 과정에서 이전에 계산된 그래디언트 값들이 누적되는 경우가 있을 수 있습니다. 이를 방지하기 위해 매 학습 반복(iteration)마다 그래디언트를 초기화해주어야 합니다.
# model.zero_grad()는 모델의 parameters() 메서드를 통해 반환된 모든 매개변수들의 그래디언트를 0으로 설정합니다. 
# 이를 통해 다음 학습 반복에서 새로운 그래디언트 값을 계산할 수 있습니다.

for epoch_i in range(0, epochs): 
  print('---------- Epoch {:} / {:} ----------'.format(epoch_i + 1, epochs))
  t0 = time.time() # 현재 epoch의 시작 시간을 기록
  total_loss = 0 #  현재 epoch의 총 손실(loss) 값을 초기화
  total_correct = 0  # 정확한 예측의 개수를 추적하기 위한 변수
  
  model.train() #  모델을 학습 모드로 설정

  for step, batch in tqdm(enumerate(train_dataloader)): 
    if step % 500 == 0 and not step == 0: 
      elapsed = format_time(time.time() - t0)
      print(' Batch {:>5,} of {:>5,}, Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
    batch = tuple(t.to(device) for t in batch)     
    b_input_ids, b_input_mask, b_labels = batch

    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    # model은 BERT 모델 객체
    #  attention_mask는 패딩을 위한 마스크를 나타내는 인자
    #  labels는 모델의 출력과 비교하여 손실을 계산하기 위한 정답 레이블
    loss = outputs[0]
    total_loss += loss.item()
    loss.backward()
    
    # 정확도 계산
    logits = outputs[1]  # 모델의 예측값 (로짓)
    preds = torch.argmax(logits, dim=1)  # 가장 높은 로짓 값을 갖는 클래스로 예측
    total_correct += torch.sum(preds == b_labels).item()  # 정확한 예측의 개수 계산

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) 
    # threshold 초과시, gradient clipping 해줌 
    optimizer.step()
    scheduler.step()

    model.zero_grad()

  avg_train_loss = total_loss / len(train_dataloader)
  accuracy = total_correct / len(train_dataloader.dataset)  # 전체 데이터셋에 대한 정확도 계산


  print("")
  print("Average training loss: {0:.4f}".format(avg_train_loss))
  print("Training epoch taken: {:}".format(format_time(time.time() - t0)))
  print("Accuracy: {:.2%}".format(accuracy))


---------- Epoch 1 / 5 ----------


501it [01:22,  6.08it/s]

 Batch   500 of   825, Elapsed: 0:01:22.


825it [02:15,  6.10it/s]



Average training loss: 0.5078
Training epoch taken: 0:02:15
Accuracy: 78.41%
---------- Epoch 2 / 5 ----------


...


Average training loss: 0.1865
Training epoch taken: 0:02:13
Accuracy: 93.14%
---------- Epoch 5 / 5 ----------


501it [01:21,  6.27it/s]

 Batch   500 of   825, Elapsed: 0:01:21.


825it [02:13,  6.20it/s]


Average training loss: 0.1373
Training epoch taken: 0:02:13
Accuracy: 95.21%


### 검증 데이터에 대한 평가 

In [ ]:
t0 = time.time()
# eval() : 파이썬에서 문자열로 된 코드 실행 함수 
# eval("1+2")
model.eval() 
accum_logits, accum_label_ids = [], []

for batch in validation_dataloader: 
  batch = tuple(t.to(device) for t in batch) 
  b_input_ids, b_input_mask, b_labels = batch
  #  validation_dataloader에서 반복하면서 검증 데이터 배치를 가져오고, 각 텐서를 GPU로 이동
  # (to(device)를 사용하여 디바이스를 지정

  with torch.no_grad(): 
    outputs = model(b_input_ids, 
                    token_type_ids=None,
                    attention_mask = b_input_mask)
    
    # print(outputs[0])

    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    for b in logits:
      # 3개의 값 중 가장 큰 값을 예측한 인덱스로 결정 
      # 예를 들어 [3.5134, - 0.308765, -2.11316] >> 0
      accum_logits.append(np.argmax(b))

    for b in label_ids: 
      accum_label_ids.append(b)
  
accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)
results = metrics(accum_logits, accum_label_ids)

print(accum_logits)
print(accum_label_ids)
print(results)

print("Accuracy: {0:.4f}".format(results['accuracy']))
print("f1_macro score: {0:.4f}".format(results['f1_macro']))
print("f1_micro score: {0:.4f}".format(results['f1_micro']))
print("f1_weighted score: {0:.4f}".format(results['f1_weighted']))

[1 1 0 ... 1 1 1]
[1 1 0 ... 1 1 1]
{'accuracy': 0.8129452781567379, 'f1_macro': 0.8190171966085233, 'f1_micro': 0.8129452781567379, 'f1_weighted': 0.8128970229544441}
Accuracy: 0.8129
f1_macro score: 0.8190
f1_micro score: 0.8129
f1_weighted score: 0.8129


### 모델 저장과 모델 로드 

In [ ]:
# 폴더 생성 
%mkdir model 

���� ���͸� �Ǵ� ���� model��(��) �̹� �ֽ��ϴ�.


In [ ]:
pwd

'c:\\Users\\SBAuser\\Desktop\\Team_project_Toon_NLP\\Team_project_Toon_NLP'

In [ ]:
path = './model/'

In [ ]:
# 모델 저장 
torch.save(model.state_dict(), "./model/bert_multi_sentiment.pt")



In [4]:
# 모델 로드 
model.load_state_dict(torch.load("./bert_multi_sentiment.pt"))

NameError: name 'model' is not defined

### 테스트 데이터에 대한 평가 

In [ ]:
t0 = time.time()
model.eval()
accum_logits, accum_label_ids = [],[]

for step, batch in tqdm(enumerate(test_dataloader)):
  if step % 100 == 0 and not step == 0: 
    elapsed = format_time(time.time() - t0)
    print('Batch {:>5,} of {:>5,}, Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

  batch = tuple(t.to(device) for t in batch)
  b_input_ids, b_input_mask, b_labels = batch

  with torch.no_grad():
    outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask = b_input_mask)
  logits = outputs[0]
  logits = logits.detach().cpu().numpy()  
  label_ids = b_labels.to('cpu').numpy()

  for b in logits: 
    accum_logits.append(np.argmax(b))

  for b in label_ids: 
    accum_label_ids.append(b)

accum_logits = np.array(accum_logits)
accum_label_ids = np.array(accum_label_ids)
results = metrics(accum_logits, accum_label_ids)

print(accum_logits)
print(accum_label_ids)
print(results)

print("Accuracy: {0:.4f}".format(results['accuracy']))
print("f1_macro score: {0:.4f}".format(results['f1_macro']))
print("f1_micro score: {0:.4f}".format(results['f1_micro']))
print("f1_weighted score: {0:.4f}".format(results['f1_weighted']))


102it [00:04, 22.97it/s]

Batch   100 of   258, Elapsed: 0:00:04.


204it [00:08, 22.98it/s]

Batch   200 of   258, Elapsed: 0:00:09.


258it [00:11, 22.95it/s]

[2 0 2 ... 2 0 0]
[1 0 1 ... 2 0 0]
{'accuracy': 0.8114237205918021, 'f1_macro': 0.8160182756587008, 'f1_micro': 0.8114237205918021, 'f1_weighted': 0.8117379977061263}
Accuracy: 0.8114
f1_macro score: 0.8160
f1_micro score: 0.8114
f1_weighted score: 0.8117


### 예측 

In [ ]:
# from transformers import pipeline
# pipeline : 정형화된 데이터 일 경우 사용 

# return_all_scores=True : 모든 라벨에 대한 확률값 반환 
# pipe = pipeline('text-classification', model = model.cuda(), tokenizer = tokenizer, device=0, max_length=512, 
#                 return_all_scores=True, function_to_apply='softmax' )
# pipe
# result = pipe('화난다')
# print(result)

c:\Users\SBAuser\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\pipelines\text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
from transformers import pipeline
pipe = pipeline('text-classification', model = model.cuda(), tokenizer = tokenizer, device=0, max_length=512, 
                 function_to_apply='softmax')

In [ ]:
result = pipe('SK하이닉스가 매출이 급성장하였다')
print(result)

[{'label': 'LABEL_0', 'score': 0.9661669135093689}]


In [ ]:
label_dict = {'LABEL_0': '기쁨', 'LABEL_1':'슬픔', 'LABEL_2':'분노'}
label_dict['LABEL_0']

In [ ]:
def prediction(text): 
   result = pipe(text)

   return label_dict[result[0]['label']]

In [ ]:
prediction("화난다")

TypeError: list indices must be integers or slices, not str

In [ ]:
prediction("나 너무 슬퍼 ㅠㅠ")

['슬픔']

In [ ]:
prediction("와 기분이 너무 좋아!!")

['기쁨']

In [ ]:
prediction("오늘 행복해 ")

['기쁨']

In [ ]:
prediction("시험을 망쳤어 우울해")

['슬픔']

In [ ]:
result = pipe('호미리 깨물어 주고 싶어')
print(result)

[{'label': 'LABEL_2', 'score': 0.9934315085411072}]
